In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Redes Convolucionales: Introducción

Vamos a comenzar mostrando una *Red Convolucional* muy simple para abordar el problema de clasificación MNIST. Con lo que hemos visto hasta ahora no resultará tan extraño, y más adelante detallaremos cada una de las capas que lo componen describiendo la funcionalidad que juegan en la red global. Veremos que, aunque la red convolucional que construiremos de forma directa es muy simple, supera el rendimiento de la red clásica que creamos en el primer notebook.

Como se puede observar en el siguiente código, esencialmente la red convolucional está formada por dos capas convoluciones bidimensionales `layer_conv_2d` seguidas de dos capas max_pooling `layer_max_pooling_2d`.


In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Veamos un resumen de la arquitectura que hemos construido:

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Una red convolucional toma como dato de entrada un tensor de la forma `(image_height, image_width, image_channels)`. En este caso, para ajustarse a las características de las imágenes de MNIST, será `(28, 28, 1)`. Observa que el tamaño del dato de entrada se pasa como argumento de la primera capa con `input_shape = (28, 28, 1)`.

Observa que cada capa de tipo `layer_conv_2d()` y `layer_max_pooling_2d()` dan como salida un tendos 3D tensor de forma `(height, width, channels)`. Tanto la anchura como altura del tensor tiende a disminuir a medida que avanzamos en la red. El número de canales se controla por medio del primer argumento que se le pasa a las capas `layer_conv_2d()` (que en este caso es 32 o 64).

El siguiente paso es pasar la salida de la última capa anterior (de tamaño `(3, 3, 64)`) a una red densa clasificadora similar a las que ya hemos visto en ejemplos anteriores. Como estas capas procesan vectores, que son 1D, y la entrada es un tensor3D, lo primero que hemos de hacer es aplanar el tensor por medio de la capa `layer_flatten()`, que proporciona directamente Keras:

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Como el objetivo es calcular una clasificación de 10 clases, la última capa es una capa densa con 10 unidades y con salida softmax. La red completa queda por tanto:


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Una vez definida la red, realizamos el entrenamiento de forma similar a como hicimos en el modelo simple de MNIST:


In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 8s - loss: 0.1766 - acc: 0.9440     
Epoch 2/5
60000/60000 [==============================] - 7s - loss: 0.0462 - acc: 0.9855     
Epoch 3/5
60000/60000 [==============================] - 7s - loss: 0.0322 - acc: 0.9902     
Epoch 4/5
60000/60000 [==============================] - 7s - loss: 0.0241 - acc: 0.9926     
Epoch 5/5
60000/60000 [==============================] - 7s - loss: 0.0187 - acc: 0.9943     


Tras el entrenamiento podemos evaluar el modelo sobre los datos de test:


In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [9]:
test_acc

0.99129999999999996

Donde comprobamos que alcanzamos una del 99%, un incremento relativo del 68% sobre el modelo básico visto anteriormente.